In [1]:
# Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist, pdist, squareform
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import rbf_kernel

In [3]:
def get_data(path):
    '''
    --------------------------
    Load data from source
    Input: Path to data
    Output: Data-frame
    -------------------------
    '''
    df = pd.read_csv(path)
    return(df)

In [4]:
def train_test_split(df, train_percent=0.666666):
    '''
    --------------------------
    Load data from source
    Input: Path to data
    Output: Data-frame
    -------------------------
    '''
    np.random.seed(4109)
    df = df.sample(frac=1).reset_index(drop=True)
    
    n_train = round(train_percent*len(df))
    df_train = df[:n_train]
    df_test = df[n_train:]
    
    return(df_train, df_test)

In [5]:
def get_grid_search_vals(start_g, stop_g, start_s, stop_s):
    '''
    --------------------------
    Load data from source
    Input: Path to data
    Output: Data-frame
    -------------------------
    '''
    gamma_powers = np.arange(start_g, stop_g + 1, 1)
    sigma_powers = np.arange(start_s, stop_s + 1, 0.5)
    
    gamma = np.power(0.5, gamma_powers)
    sigma = np.power(0.5, sigma_powers)
    
    return(gamma, sigma)

In [40]:
def get_pairwise_dist(A, B):
    '''
    --------------------------
    Load data from source
    Input: Path to data
    Output: Data-frame
    
    References for einsum:
    https://ajcr.net/Basic-guide-to-einsum/
    https://stackoverflow.com/questions/42991347/how-to-find-the-pairwise-differences-between-rows-of-two-very-large-matrices-usi
    -------------------------
    '''
    dist = np.einsum('ij,ij->i',A, A)[:,None] + np.einsum('ij,ij->i',B,B) - 2*np.dot(A,B.T)
    return(dist)

In [35]:
def test_pairwise_dist(A, B):
    '''
    --------------------------
    Load data from source
    Input: Path to data
    Output: Data-frame
    
    References for einsum:
    https://ajcr.net/Basic-guide-to-einsum/
    https://stackoverflow.com/questions/45896939/using-python-numpy-einsum-to-obtain-dot-product-between-2-matrices
    -------------------------
    '''
    dist = get_pairwise_dist(A, B)
    dist_sk = cdist(A, B)**2
    return(dist, dist_sk, np.allclose(dist, dist_sk))

In [85]:
def get_gaussian_kernel(X, X_test, sigma):
    '''
    --------------------------
    Load data from source
    Input: Path to data
    Output: Data-frame
    -------------------------
    '''
    scale = -0.5/np.power(sigma, 2)
    K = get_pairwise_dist(X, X_test) 
    K = K*scale
    K = np.exp(K)
    return(K)

In [80]:
def test_gaussian_kernel(X, X_test, sigma):
    '''
    --------------------------
    Load data from source
    Input: Path to data
    Output: Data-frame
    -------------------------
    '''
    kernel = get_gaussian_kernel(X, X_test, sigma)
    kernel_sk = rbf_kernel(X, X_test, 0.5)
    return(np.allclose(kernel, kernel_sk))

In [108]:
def get_kernel_ridge_coeff(X, Y, sigma, gamma):
    '''
    --------------------------
    Load data from source
    Input: Path to data
    Output: Data-frame
    -------------------------
    '''
    m = Y.shape[0]
    K = get_gaussian_kernel(X, X, sigma)
    alpha = np.linalg.solve(K + gamma*np.identity(m), Y)
    return(alpha)

In [113]:
def test_kernel_ridge_coeff():
    '''
    --------------------------
    Load data from source
    Input: Path to data
    Output: Data-frame
    -------------------------
    '''
    a = np.array([1, 2, 3, 4, 5, 6, 7, 8]).reshape(2,4)
    b = np.array([2, 4]).reshape(2,1)
    result = get_kernel_ridge_coeff(K, b, 1, 2)
    return(result)

In [162]:
def get_kernel_ridge_prediction(alpha, X, X_test, sigma):
    '''
    --------------------------
    Load data from source
    Input: Path to data
    Output: Data-frame
    -------------------------
    '''
    K_test = get_gaussian_kernel(X, X_test, sigma)
    y_pred = np.einsum('ij,ij->j', alpha, K_test)
    return(y_pred)

In [153]:
def test_kernel_ridge_prediction():
    '''
    --------------------------
    Load data from source
    Input: Path to data
    Output: Data-frame
    -------------------------
    '''
    a = np.array([1, 2, 3, 4, 5, 6, 7, 8]).reshape(2,4)
    b = np.array([5, 8, 9, 10, 11, 12, 13, 14, 18, 19, 20, 21, 25, 26, 7, 8]).reshape(4,4)
    
    alpha = test_kernel_ridge_coeff()
    result = get_kernel_ridge_prediction(alpha, a, b, 1)
    
    return(result)

In [154]:
def get_k_folds(df_train, k = 5):
    '''
    --------------------------
    Load data from source
    Input: Path to data
    Output: Data-frame
    -------------------------
    '''
    # Set the seed to make sure reproducible
    # Then create a list of folds
    np.random.seed(43890)
    fold_size = round(len(df_train)/k)
    df_train = df_train.sample(frac=1).reset_index(drop=True)
    df_folds = [df_train.iloc[i:i+fold_size] for i in range(0,len(df_train)-fold_size+1,fold_size)]
    return(df_folds)

In [155]:
def get_mse(y, y_hat):
    '''
    --------------------------
    Load data from source
    Input: Path to data
    Output: Data-frame
    -------------------------
    '''
    return((1/y.shape[0])*np.sum(np.power((y-y_pred),2)))

In [156]:
def plot_mse(gamma, sigma):
    '''
    --------------------------
    Load data from source
    Input: Path to data
    Output: Data-frame
    -------------------------
    '''
    pass

In [157]:
def get_best_params(history):
    '''
    --------------------------
    Load data from source
    Input: Path to data
    Output: Data-frame
    -------------------------
    '''
    best_result = history['val_mse'].argmin()
    best_alpha, best_sigma = history[best_result]['alpha'], history[best_result]['sigma']
    best_mse = history['val_mse'].min()
    
    return(best_result, best_mse, best_params)

In [158]:
def train_kernel_ridge(X_train, X_val, Y_train, Y_val, sigma, gamma):
    '''
    --------------------------
    Load data from source
    Input: Path to data
    Output: Data-frame
    -------------------------
    '''
    # Get dual form estimates
    alpha = get_kernel_ridge_coeff(x, y, sigma, gamma)
    
    # Get predictions
    y_train_preds = get_kernel_ridge_prediction(alpha, X_train, X_train, sigma)
    y_val_preds = get_kernel_ridge_predictions(alpha, X_train, X_train, sigma)
    
    # Get loss values
    mse_train = get_mse(Y_train, y_pred_train)
    mse_val = get_mse(Y_val, y_pred_val)
    
    return(mse_train, mse_val)

In [ ]:
def tests():
    '''
    --------------------------
    Load data from source
    Input: Path to data
    Output: Data-frame
    -------------------------
    '''
    # Make test data
    a = np.array([1, 2, 3, 4, 5, 6, 7, 8]).reshape(2,4)
    b = np.array([2, 4, 6, 8, 10, 12, 14, 16]).reshape(2,4)
    
    dist_test_result = test_pairwise_dist(a, b)
    kern_test_result = test_gaussian_kernel(a, b, 1)
    
    coeff_test_result = test_kernel_ridge_coeff()
    prediction_test_result = test_kernel_ridge_prediction()
    
    return(dist_test_result, kern_test_result)

In [159]:
def main():
    '''
    --------------------------
    Load data from source
    Input: Path to data
    Output: Data-frame
    -------------------------
    '''
    # Setup
    path = 'http://www0.cs.ucl.ac.uk/staff/M.Herbster/boston-filter/Boston-filtered.csv'
    
    # Get list of parameters to search over
    gamma_list, sigma_list = get_grid_search_vals(26, 40, 7, 13)
    
    # Load data
    df = get_data(path)
    
    # Split into train and test: we are making 2/3 splits as required by default
    df_train, df_test = train_test_split(df)
    
    # Divide training data into k=5 folds for cross-validation as question asks
    df_folds = get_k_folds(df_train)
    
    # Convert data-frames into numpy arrays
    X_folds = [df_fold[:,:12].to_numpy() for df_fold in df_folds]
    Y_folds = [df_fold['MEDV'].to_numpy() for df_fold in df_folds]

    # Create a container to store results
    history = {'gamma': [],
               'sigma': [],
               'avg_mse': []}